### Universidad Nacional de Colombia
### Diplomado Ciencia de datos

### Accidentes de tránsito en New York 

En este caso práctico haremos la exploración de un data set aplicando  algunas de las etapas de los procesos de ciencia de datos. Haremos transformación y limpieza de los datos. Entenderemos cómo extraer valor de los datos desde una perspectiva exploratoria

El caso estará estructurado así

1. Explorar la estructura de los datos
2. Hacer un análisis exploratorio y descriptivo de las principales variables
3. Sacar conclusiones e hipótesis de análisis a partir de los hallazgos

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy

In [ ]:
# display setting Para visualizar el máximo de columnas
pd.set_option('display.max_columns', None)

**Contexto** 
New York ha tenido un incremento en el número de accidentes de tránsito constante y se requiere analizar datos de estos accidentes del periodo Enero 2018 - Agosto 2019. 

**Problema de negocio**
Identificar patrones en la data que permita tomar decisiones informadas dirigidas hacia la planeación de políticas públicas de caracter preventivo para diminuir el número y/o gravedad de los accidentes

In [ ]:
with open('accidents.csv') as f:
    accidentes=pd.read_csv(f, delimiter=';')

In [ ]:
accidentes.head()

Se tienen las variables 
- BOROUGH: Vecindario donde ocurrió el accidente
- COLLISION_ID: ID del accidente
- CONTRIBUTING FACTOR VEHICLE (1, 2, 3, 4, 5): Razones del accidente
- CROSS STREET NAME: La calle cruzada más cercana en la que pasó el accidente
- DATE: Fecha del accidente
 -TIME: Hora del accidente
 -LATITUDE: Latitud del accidente
- LONGITUDE: Longitud del accidente
- NUMBER OF (CYCLISTS, MOTORISTS, PEDESTRIANS) INJURED: Número de heridos de cada tipo
- NUMBER OF (CYCLISTS, MOTORISTS, PEDESTRIANS) KILLED: Número de muertos de cada tipo
- ON STREET NAME: Calle de accidente
- VEHICLE TYPE CODE (1, 2, 3, 4, 5): Tipo de vehículo involucrado en el accidente
- ZIP CODE: Código zip del accidente

In [ ]:
accidentes.shape

**¿Han incrementado el número de accidentes en el periodo de observación?**

Para hacer éste análisis primero debemos cambiar el formato de la columna de la fecha para manejarla apropiadamente. 

In [ ]:
accidentes['DATE']=pd.to_datetime(accidentes['DATE'])

Con esta transformación podemos extraer el mes de la fecha y hacer un análisis por meses. ¿Qué podemos concluir del número de accidentes por mes?

In [ ]:
accidentes['MES'] = accidentes['DATE'].dt.to_period('M')
accidentes_mes = accidentes.groupby(accidentes['MES']).size()
accidentes_mes.plot.line()

**¿Existen patrones horarios en los accidentes?**

In [ ]:
# transformación de la hora del accidente a formato de fecha
accidentes['TIME'] = pd.to_datetime(accidentes['TIME'])

Al igual que la fecha podemos transformar el formato de la hora y crear una variable que contenga únicamente la hora del accidente

In [ ]:
accidentes['HORA'] = accidentes['TIME'].dt.hour

In [ ]:
# Conteo de accidentes por hora
accidentes_hora = accidentes.groupby('HORA').size()
accidentes_hora.plot.bar()

**¿Existen patrones en los accidentes según el día de la semana?**

In [ ]:
# Creamos una variable con el día de la semana de la fecha del accidente
accidentes['DIA'] = accidentes['DATE'].dt.weekday

In [ ]:
accidentes.head()

In [ ]:
# Conteo de accidentes por dia de la semana
accidentes_dia = accidentes.groupby('DIA').size()
accidentes_dia.plot.bar()

**¿Existen patrones accidentales por vecindario?**

In [ ]:
vec = accidentes.groupby('BOROUGH').size()
vec.plot.bar()

## ¿Por qué razón deberíamos estandarizar estos conteos?

Carguemos la información adicional de los vecindarios para poder analizarlos

In [ ]:
borough_data = pd.read_csv('borough_data.csv')

In [ ]:
borough_data

Para hacer los conteos de accidentes comparables podemos relativizar utilizando el área de los vecindarios. Para esto primero debemos asociar las dos tablas de información

In [ ]:
print(accidentes['BOROUGH'].unique())

El único vecindario que no haría match sería 'the bronx' con 'BRONX'

In [ ]:
borough_data.loc[borough_data.borough=='the bronx','borough'] = 'bronx'

In [ ]:
borough_frame = pd.DataFrame(vec)
borough_frame.columns = ['count']
borough_frame['borough'] = borough_frame.index
borough_frame

Adicionalmente, necesitamos transformar los valores en el df a minúsculas para poder calcular los conteos de accidentes relativos al área del vecindario

In [ ]:
borough_data.borough = borough_data.borough.str.upper()
borough_data

In [ ]:
borough_frame = borough_frame.merge(borough_data, left_on='borough',right_on='borough')
borough_frame

In [ ]:
borough_frame['accidentes_por_area'] = borough_frame['count']/borough_frame['area']
borough_frame.plot.bar(x='borough', y='accidentes_por_area')

**Ampliemos el análisis de manera bivariada**

¿Existen patrones horarios en la accidentalidad por vecindario?

In [ ]:
# creamos conteo de accidentes por hora y vecindario
df1 = pd.DataFrame({'count': accidentes.groupby(['BOROUGH', 'HORA']).size()})
df1 = df1.reset_index()

In [ ]:
plt.figure(figsize=(10,8))
chart = sns.FacetGrid(df1, col='BOROUGH', margin_titles=True, col_wrap=3, aspect=2, row_order=accidentes['BOROUGH'].unique)
chart.map(sns.barplot, 'HORA', 'count',)

**¿Cuáles son las causas más comunes de los accidentes?**

Debemos tener en cuenta las 5 columnas de CONTRIBUTING FACTOR para hacer este análisis

In [ ]:
columns = ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 
           'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
          'CONTRIBUTING FACTOR VEHICLE 5']

In [ ]:
# creamos un nuevo df con todos los factores en una sola columna
frames=[accidentes[column].reset_index().rename(columns={column:'FACTOR'}) for column in columns]
factors_df=pd.concat(frames)


In [ ]:
# agurpamos por el índice para poder descartar factores que se repiten en el mismo accidente
no_rep_factors=factors_df.groupby(['index','FACTOR']).size().reset_index()

In [ ]:
# Conteo de accidentes por factor
final_df=no_rep_factors.groupby(['FACTOR']).size()
final_df=final_df.sort_values(ascending=False).reset_index()
final_df.head(10)

**Y ¿cuáles son los tipos de vehículos involucrados en mas accidentes?**

In [ ]:
columns = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
           'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5',]

#First, we concatenate all the corresponding columns into a big data-frame
frames=[accidentes[column].reset_index().rename(columns={column:'VEHICLE'}) for column in columns]
vehicle_df=pd.concat(frames)


#The reasoning behind doing reset_index() is that we can use a groupby on the indexes to delete repetitions as follows:
no_rep_vehicle=vehicle_df.groupby(['index','VEHICLE']).size().reset_index()

In [ ]:

#The data frame we are looking for is given by
final_dfv=no_rep_vehicle.groupby(['VEHICLE']).size()

#Since we want to search for the most common factors of accidents, we can do the following:
final_dfv=final_dfv.sort_values(ascending=False).reset_index()
final_dfv.head(10)


**Pueden existir patrones de factores o vehículos diferencial para vecindarios?**

In [ ]:
columns =['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 
           'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
          'CONTRIBUTING FACTOR VEHICLE 5']


# Concatenate all corresponding frames.
frames=[accidentes[['BOROUGH',column]].rename(columns={'BOROUGH':'BOROUGH',column:'FACTOR'}) for column in columns]
factor_df=pd.concat(frames).reset_index()

#Count repetitions
no_rep_factor=factor_df.groupby(['index','BOROUGH','FACTOR']).size().reset_index()


final_df=no_rep_factor.groupby(['BOROUGH','FACTOR']).size().sort_values(ascending=False).reset_index()

final_df.head(10)

In [ ]:
columns = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
           'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5',]

# Concatenate all corresponding frames.
frames=[accidentes[['BOROUGH',column]].rename(columns={'BOROUGH':'BOROUGH',column:'VEHICLE'}) for column in columns]
vehicles_df=pd.concat(frames).reset_index()

#Count repetitions
no_rep_vehicles=vehicles_df.groupby(['index','BOROUGH','VEHICLE']).size().reset_index()

#Then get rid of these
final_df=no_rep_vehicles.groupby(['BOROUGH','VEHICLE']).size().sort_values(ascending=False).reset_index()

final_df.head(10)

## Ejercicios

**1. ¿Qué podemos decir sobre el número de personas heridas y muertes en los accidentes? Exploremos el comportamiento del total de heridos y total de muertes en todos los accidentes. (Crear una nueva columna que sume todos los herido y otra que sume todas las muertes)**

**2. Exploremos las muertes y heridas por peatón, ciclista y motociclista. Existen diferencias en estos? En qué casos mueren más personas durante los accidentes? En qué casos resultan más heridos?**

**3. ¿ Existen patrones en las muertes y heridos en accidentes por vecindario? En cuál vecindario se presentaron más muertes? En cuál menos?**

**4. A partir de los análisis anteriores, qué recomendaciones sobre la planeación de políticas preventivas frente a la accidentalidad se podrían hacer para reducir los accidentes y su gravedad?**

**5. ¿Qué otras variables o análisis exploratorios podríamos hacer para complementar este análisis?**

**6. ¿Cuáles podrían ser los siguientes pasos para el análisis de este problema en búsqueda de soluciones?**

## Conclusiones

- La exploración y descripción inicial de los datos es una gran herramienta para encontrar patrones de interés del problema 
- Encontramos patrones en los datos que ayudan a generar hipótesis sobre el comportamiento de los accidentes
- Resultan ser relevantes factores cómo el día de la semana, hora y vecindario para analizar el comportamiento de la accidentalidad
- Esta descripción puede ser una guía importante para definir pasos siguientes en un análisis con mayor profundidad